# Mask R-CNN Demo

A quick intro to using the pre-trained model to detect and segment objects.

In [1]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images/images_10fps")

Using TensorFlow backend.


## Configurations

We'll be using a model trained on the MS-COCO dataset. The configurations of this model are in the ```CocoConfig``` class in ```coco.py```.

For inferencing, modify the configurations a bit to fit the task. To do so, sub-class the ```CocoConfig``` class and override the attributes you need to change.

In [2]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    BATCH_SIZE = GPU_COUNT * IMAGES_PER_GPU

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.3
DETECTION_NMS_THRESHOLD        0.5
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  3840
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  1024
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              pad64
IMAGE_SHAPE                    [3840 3840    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

## Create Model and Load Trained Weights

In [3]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

Non-maximum supression threshold:0.5


## Class Names

The model classifies objects and returns class IDs, which are integer value that identify each class. Some datasets assign integer values to their classes and some don't. For example, in the MS-COCO dataset, the 'person' class is 1 and 'teddy bear' is 88. The IDs are often sequential, but not always. The COCO dataset, for example, has classes associated with class IDs 70 and 72, but not 71.

To improve consistency, and to support training on data from multiple sources at the same time, our ```Dataset``` class assigns it's own sequential integer IDs to each class. For example, if you load the COCO dataset using our ```Dataset``` class, the 'person' class would get class ID = 1 (just like COCO) and the 'teddy bear' class is 78 (different from COCO). Keep that in mind when mapping class IDs to class names.

To get the list of class names, you'd load the dataset and then use the ```class_names``` property like this.
```
# Load COCO dataset
dataset = coco.CocoDataset()
dataset.load_coco(COCO_DIR, "train")
dataset.prepare()

# Print class names
print(dataset.class_names)
```

We don't want to require you to download the COCO dataset just to run this demo, so we're including the list of class names below. The index of the class name in the list represent its ID (first class is 0, second is 1, third is 2, ...etc.)

In [4]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

## Run Object Detection

In [ ]:
# Load a random image from the images folder
#file_names = next(os.walk(IMAGE_DIR))[2]
#image = skimage.io.imread(os.path.join(IMAGE_DIR, random.choice(file_names)))

# Run detection
#results = model.detect([image], verbose=1)

# Visualize results
#r = results[0]
#visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'])

In [ ]:
import glob
import time

count=0

timestr = time.strftime("%Y%m%d-%H%M%S")
log_file = open('log_' + timestr + '.txt', 'a')

for filename in sorted(glob.glob(os.path.join(IMAGE_DIR,'*.jpg'))):
    print(filename)
    image = skimage.io.imread(filename)
    # Run detection
    start = time.time()
    results = model.detect([image], verbose=1)
#    results = model.detect([image,image], verbose=1)
    # Visualize results
    end = time.time()
    print(end - start)
    r = results[0]
    visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'], count, log_file)
    count=count+1
    
log_file.close()

/home/dmitriy.khvan/Mask_RCNN/images/images_10fps/thumb00001.jpg
1
Processing 1 images
image                    shape: (2160, 3840, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 2176, 3840, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 3840.00000  int64
anchors                  shape: (1, 2086920, 4)       min:   -0.16645  max:    1.13703  float32
11.079881429672241
/home/dmitriy.khvan/Mask_RCNN/images/images_10fps/thumb00002.jpg
1
Processing 1 images
image                    shape: (2160, 3840, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 2176, 3840, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 3840.00000  int64
anchors                  shape: (1, 2086920, 4)       min:   -0.16645  max:    1.13703  float32
1.4593632221221924
/home/dmitriy.khvan/Mask_RCN

1.4198362827301025
/home/dmitriy.khvan/Mask_RCNN/images/images_10fps/thumb00019.jpg
1
Processing 1 images
image                    shape: (2160, 3840, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 2176, 3840, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 3840.00000  int64
anchors                  shape: (1, 2086920, 4)       min:   -0.16645  max:    1.13703  float32
1.484464406967163
/home/dmitriy.khvan/Mask_RCNN/images/images_10fps/thumb00020.jpg
1
Processing 1 images
image                    shape: (2160, 3840, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 2176, 3840, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 3840.00000  int64
anchors                  shape: (1, 2086920, 4)       min:   -0.16645  max:    1.13703  float32
1.4331762790679932
/home/dmit

/home/dmitriy.khvan/Mask_RCNN/mrcnn/visualize.py:114: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  _, ax = plt.subplots(1, figsize=figsize)


/home/dmitriy.khvan/Mask_RCNN/images/images_10fps/thumb00023.jpg
1
Processing 1 images
image                    shape: (2160, 3840, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 2176, 3840, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 3840.00000  int64
anchors                  shape: (1, 2086920, 4)       min:   -0.16645  max:    1.13703  float32
1.4705348014831543
/home/dmitriy.khvan/Mask_RCNN/images/images_10fps/thumb00024.jpg
1
Processing 1 images
image                    shape: (2160, 3840, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 2176, 3840, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 3840.00000  int64
anchors                  shape: (1, 2086920, 4)       min:   -0.16645  max:    1.13703  float32
1.408776044845581
/home/dmitriy.khvan/Mask_RCNN

1.3948192596435547
/home/dmitriy.khvan/Mask_RCNN/images/images_10fps/thumb00040.jpg
1
Processing 1 images
image                    shape: (2160, 3840, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 2176, 3840, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 3840.00000  int64
anchors                  shape: (1, 2086920, 4)       min:   -0.16645  max:    1.13703  float32
1.3978407382965088
/home/dmitriy.khvan/Mask_RCNN/images/images_10fps/thumb00041.jpg
1
Processing 1 images
image                    shape: (2160, 3840, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 2176, 3840, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 3840.00000  int64
anchors                  shape: (1, 2086920, 4)       min:   -0.16645  max:    1.13703  float32
1.435112476348877
/home/dmit

...

1.4443223476409912
/home/dmitriy.khvan/Mask_RCNN/images/images_10fps/thumb00244.jpg
1
Processing 1 images
image                    shape: (2160, 3840, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 2176, 3840, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 3840.00000  int64
anchors                  shape: (1, 2086920, 4)       min:   -0.16645  max:    1.13703  float32
1.4377837181091309
/home/dmitriy.khvan/Mask_RCNN/images/images_10fps/thumb00245.jpg
1
Processing 1 images
image                    shape: (2160, 3840, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 2176, 3840, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 3840.00000  int64
anchors                  shape: (1, 2086920, 4)       min:   -0.16645  max:    1.13703  float32
1.4377894401550293
/home/dmi

1.4466214179992676
/home/dmitriy.khvan/Mask_RCNN/images/images_10fps/thumb00261.jpg
1
Processing 1 images
image                    shape: (2160, 3840, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 2176, 3840, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 3840.00000  int64
anchors                  shape: (1, 2086920, 4)       min:   -0.16645  max:    1.13703  float32
1.3812484741210938
/home/dmitriy.khvan/Mask_RCNN/images/images_10fps/thumb00262.jpg
1
Processing 1 images
image                    shape: (2160, 3840, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 2176, 3840, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 3840.00000  int64
anchors                  shape: (1, 2086920, 4)       min:   -0.16645  max:    1.13703  float32
1.393047571182251
/home/dmit

1.4310047626495361
/home/dmitriy.khvan/Mask_RCNN/images/images_10fps/thumb00278.jpg
1
Processing 1 images
image                    shape: (2160, 3840, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 2176, 3840, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 3840.00000  int64
anchors                  shape: (1, 2086920, 4)       min:   -0.16645  max:    1.13703  float32
1.4373829364776611
/home/dmitriy.khvan/Mask_RCNN/images/images_10fps/thumb00279.jpg
1
Processing 1 images
image                    shape: (2160, 3840, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 2176, 3840, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 3840.00000  int64
anchors                  shape: (1, 2086920, 4)       min:   -0.16645  max:    1.13703  float32
1.4297590255737305
/home/dmi

1.4676175117492676
/home/dmitriy.khvan/Mask_RCNN/images/images_10fps/thumb00295.jpg
1
Processing 1 images
image                    shape: (2160, 3840, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 2176, 3840, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 3840.00000  int64
anchors                  shape: (1, 2086920, 4)       min:   -0.16645  max:    1.13703  float32
1.4366955757141113
/home/dmitriy.khvan/Mask_RCNN/images/images_10fps/thumb00296.jpg
1
Processing 1 images
image                    shape: (2160, 3840, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 2176, 3840, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 3840.00000  int64
anchors                  shape: (1, 2086920, 4)       min:   -0.16645  max:    1.13703  float32
1.4701039791107178
/home/dmi

1.4330708980560303
/home/dmitriy.khvan/Mask_RCNN/images/images_10fps/thumb00312.jpg
1
Processing 1 images
image                    shape: (2160, 3840, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 2176, 3840, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 3840.00000  int64
anchors                  shape: (1, 2086920, 4)       min:   -0.16645  max:    1.13703  float32
1.4780409336090088
/home/dmitriy.khvan/Mask_RCNN/images/images_10fps/thumb00313.jpg
1
Processing 1 images
image                    shape: (2160, 3840, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 2176, 3840, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 3840.00000  int64
anchors                  shape: (1, 2086920, 4)       min:   -0.16645  max:    1.13703  float32
1.4371893405914307
/home/dmi

In [ ]:
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
tf.test.is_gpu_available()

In [ ]:
 import sys
 print(sys.executable)
 print(sys.version)
 print(sys.version_info)

In [ ]:
import tensorflow